In [ ]:
from __init__ import *

In [ ]:
signal_df_ = []

for index, data in enumerate(SETTING["model_data_"]):

    signal_df = pd.read_csv(
        "../output/signal/{}.tsv".format(index), sep="\t", index_col=0
    )

    signal_df.columns.name = data["axis_1_name"]

    signal_df_.append(signal_df)

In [ ]:
if SETTING["model_mode"] == "deep":

    factorization_mode = "ws"

    sync_axis = 1

elif SETTING["model_mode"] == "range":

    factorization_mode = "hs"

    sync_axis = 0

signal_df_ = kraft.pd.sync(signal_df_, sync_axis)

In [ ]:
for r in SETTING["resolution_"]:

    matrix_ = tuple(signal_df.to_numpy() for signal_df in signal_df_)

    axis_0_label__ = tuple(signal_df.index.to_numpy() for signal_df in signal_df_)

    axis_1_label__ = tuple(signal_df.columns.to_numpy() for signal_df in signal_df_)

    axis_0_name_ = tuple(signal_df.index.name for signal_df in signal_df_)

    axis_1_name_ = tuple(signal_df.columns.name for signal_df in signal_df_)

    w_, h_, error__ = kraft.matrix_factorization.factorize(
        matrix_, factorization_mode, r
    )

    directory_path = "../output/matrix_factorization_r{}/".format(r)

    kraft.path.make(directory_path)

    kraft.matrix_factorization.plot(
        w_,
        h_,
        axis_0_label__,
        axis_1_label__,
        axis_0_name_,
        axis_1_name_,
        error__,
        directory_path=directory_path,
    )

    factor_label_ = kraft.matrix_factorization.make_factor_label_(r)

    w_df_ = tuple(
        pd.DataFrame(
            data=w,
            index=pd.Index(data=axis_0_label__[index], name=axis_0_name_[index]),
            columns=pd.Index(data=factor_label_, name="Factor"),
        )
        for index, w in enumerate(w_)
    )

    h_df_ = tuple(
        pd.DataFrame(
            data=h,
            index=pd.Index(data=factor_label_, name="Factor"),
            columns=pd.Index(data=axis_1_label__[index], name=axis_1_name_[index]),
        )
        for index, h in enumerate(h_)
    )

    file_path_template = "{}{{}}_{{}}.tsv".format(directory_path)

    for index, df in enumerate(w_df_):

        df.to_csv(file_path_template.format("w", index), sep="\t")

    for index, df in enumerate(h_df_):

        df.to_csv(file_path_template.format("h", index), sep="\t")